In [ ]:
from docx import Document

def load_and_split_docx(file_path):
    doc = Document(file_path)
    paragraphs = [p.text for p in doc.paragraphs if p.text.strip() != '']
    return paragraphs

file_path = '/root/llm/Langchain-Chatchat-master/notebooks/docx-test/docx_test.docx'
paragraphs = load_and_split_docx(file_path)
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
# for i, para in enumerate(paragraphs):
#     print(f"Paragraph {i+1}: {para}")
full_texts = read_docx(file_path)


In [13]:
print("#")

#


In [10]:
paragraphs[20]

'4.5 执行自动化验收\t17'

In [2]:
import docx
from collections import defaultdict

def extract_docx_content(file_path):
    doc = docx.Document(file_path)
    content = defaultdict(list)
    current_heading = "No Heading"
    
    for para in doc.paragraphs:
        if para.style.name.startswith('Heading'):
            current_heading = para.text
        else:
            content[current_heading].append(para.text)
    
    return content

def format_output(content):
    output = []
    for heading, paragraphs in content.items():
        block = {
            "metadata": {"title": heading},
            "content": "\n".join(paragraphs)
        }
        output.append(block)
    return output

def main(file_path):
    extracted_content = extract_docx_content(file_path)
    formatted_output = format_output(extracted_content)
    
    # Print the output (you can modify this to save to a file if needed)
    for block in formatted_output:
        print(f"Title: {block['metadata']['title']}")
        print(f"Content:\n{block['content']}\n")
        print("#"*100)

if __name__ == "__main__":
    file_path = "/root/llm/Langchain-Chatchat-master/notebooks/docx-test/docx_test.docx"  # Replace with your actual file path
    main(file_path)

Title: No Heading
Content:















目  录
1 工具概述	1
1.1 简介	1
1.2 操作流程	1
2 检查前准备	3
3 安装工具	5
3.1 场景说明	5
3.1.1 安装场景说明	5
3.1.2 巡检场景说明	5
3.2 安装MrsCare	6
3.3 对接环境	7
3.3.1 添加对接环境	7
3.3.1.1 手动添加环境信息	8
3.3.1.2 通过配置文件导入环境信息	10
3.3.2 批量添加客户端路径	11
4 检查过程	13
4.1 产品支持的任务类型、版本和用户	13
4.2 执行日常巡检检查	14
4.3 执行升级前检查	15
4.4 执行补丁前检查	16
4.5 执行自动化验收	17
4.6 执行质量验收巡检	18
4.7 执行深度巡检	19
4.8 检查结果	19
4.8.1 查看检查结果	19
4.8.2 检查项修复	20
4.8.3 重复巡检	24
4.9 多环境场景下的巡检工具部署	25
4.9.1 部署方案	25
4.9.2 部署指导	25
4.10 邮箱配置	27
4.11 巡检帮助链接	28
5 卸载工具	30
5.1 卸载SysChecker	30
5.2 卸载MrsCare	30
6 附录	32
6.1 如何修改巡检任务的超时时间	32
6.2 检查项修复指导	32
6.2.1 CDL	32
6.2.1.1 检查CDL服务和实例状态	32
6.2.1.2 检查CDL告警信息	33
6.2.1.3 检查CDL服务状态	33
6.2.2 ClickHouse	33
6.2.2.1 检查ClickHouse告警信息	33
6.2.2.2 检查ClickHouse服务健康状态	33
6.2.2.3 检查ClickHouse在ZooKeeper的quota使用情况	34
6.2.2.4 检查ClickHouse磁盘使用情况	34
6.2.2.5 检查是否有异常日志	34
6.2.2.6 检查ClickHouse服务和实例状态	34
6.2.2.7 检查ClickHouse是否支持创建MergeTree引擎表(client)	35
6.2.2.8 检查ClickHouse是否支持创建ReplacingMergeTree引擎表(client)	35
6.2.2.9 检查ClickHous

In [19]:
import docx
from collections import defaultdict
from docx.document import Document
from docx.table import Table
from docx.oxml.shape import CT_Picture
from docx.oxml.shared import qn
import base64
import io
from langchain.docstore.document import Document

def get_document_title(doc):
    # Try to get the title from document properties
    title = doc.core_properties.title
    
    # If title is not set in properties, use the first paragraph if it's in a title style
    if not title:
        if doc.paragraphs:
            first_paragraph = doc.paragraphs[0]
            if first_paragraph.style.name.lower() in ['title', 'heading 1']:
                title = first_paragraph.text
    
    # If still no title, use a default
    if not title:
        title = "Untitled Document"
    
    return title

def extract_docx_content(file_path):
    doc = docx.Document(file_path)
    content = defaultdict(list)
    
    # Get the document title and use it as the initial heading
    document_title = get_document_title(doc)
    heading_stack = [document_title]
    current_heading = document_title
    
    for element in doc.element.body:
        if element.tag.endswith('p'):
            paragraph = docx.text.paragraph.Paragraph(element, doc)
            if paragraph.style and paragraph.style.name.startswith('Heading'):
                heading_level = int(paragraph.style.name[-1])
                
                # Adjust the heading stack based on the current heading level
                while len(heading_stack) > heading_level:
                    heading_stack.pop()
                
                if len(heading_stack) < heading_level:
                    # If we jumped to a deeper level, fill in with empty headings
                    heading_stack.extend([""] * (heading_level - len(heading_stack)))
                
                heading_stack[heading_level - 1] = paragraph.text
                current_heading = document_title+" > "+" > ".join(filter(None, heading_stack))
            else:
                content[current_heading].append({"type": "text", "content": paragraph.text})
        elif element.tag.endswith('tbl'):
            table = Table(element, doc)
            table_data = []
            for row in table.rows:
                table_data.append([cell.text for cell in row.cells])
            content[current_heading].append({"type": "table", "content": table_data})
        elif element.tag.endswith('r'):
            for run in element.findall(".//"+qn('w:drawing')):
                blip = run.find(".//"+qn('a:blip'))
                if blip is not None:
                    image_rid = blip.get(qn('r:embed'))
                    image_part = doc.part.related_parts[image_rid]
                    image_bytes = image_part.blob
                    image_base64 = base64.b64encode(image_bytes).decode()
                    content[current_heading].append({"type": "image", "content": image_base64})
    
    return content

def format_output(content, chunk_size):
    output = []
    for heading, elements in content.items():
        block = {
            "metadata": {"title": heading},
            "content": []
        }
        for element in elements:
            if element['type'] == 'text':
                block['content'].append(f"{element['content']}")
            elif element['type'] == 'table':
                table_str = "\n".join([", ".join(row) for row in element['content']])
                block['content'].append(table_str)
            elif element['type'] == 'image':
                block['content'].append(f"[Base64 encoded image data]")
        block['content'] = "\n".join(block['content'])
        
        if len(block['content']) > chunk_size:
            chunks = [block['content'][i:i+chunk_size] for i in range(0, len(block['content']), chunk_size)]
            for chunk in chunks:
                output.append({"metadata": {"title": heading}, "content": chunk})
        else:    
            output.append(block)
    return output

def main(file_path, chunk_size):
    extracted_content = extract_docx_content(file_path)
    formatted_output = format_output(extracted_content, chunk_size)
    
    for block in formatted_output:
        print(block)
        print(f"Title: {block['metadata']['title']}")
        print(f"Content length: {len(block['content'])}")
        print("#" * 100)
    
    return formatted_output

if __name__ == "__main__":
    file_path = "/root/llm/Langchain-Chatchat-master/notebooks/docx-test/中国移动集中网络云承载网网管&业支CE局数据配置规范初稿（华为）V4.0.docx"
    formatted_output = main(file_path, chunk_size=8094)

{'metadata': {'title': '中国移动网络云局数据规范（华为草稿）'}, 'content': '\n\n\n\n中国移动网络云承载网网管&业支CE局数据配置规范（华为）\n\n\n\n\n\n\n\n\n\n\n版本号：4.0\n\n\n\n\n\n文档说明：本规范为中国移动网络云承载网信令CE局数据配置指导性规范。\n编号说明：\n（本文档编号依照以下原则进行：\nx.y.z    x: 大版本更新，文档结构变化\n                       y: 具体方案的更改，部分内容的修正)\n文档修改历史\n版本, 更新日期, 修改, 核定, 更新说明, 文档状态, 文档状态\n1.0, 2020-03-21, 杨应雄, 赖小勤, 征求意见稿, 初稿, 初稿\n1.1, 2020-07-08, 王永刚, 赖小勤, 删除上行接口carrier up-hold-time , 删除上行接口carrier up-hold-time , \n1.3, 2021-03, 于潇, 王永刚, 3.4.3 OSPF协议增加上行AR对接BGP和上行AR对接OSPF两种配置方式\n3.3.1删除CE-AR互联延时UP 5000, , \n4.0, 2023-04, 王永刚, , 4.4 SNMP配置将认证模式和加密算法修改为安全性更高的sha2-512和aes256, , \n, , , , , , \n\n\n\n目   录\n目   录\t4\n1 移动电信云承载网CE组网概述\t5\n1.1 设计内容\t5\n1.2 目标组网\t5\n2 软硬件版本说明\t6\n3 局数据配置规范\t7\n3.1 协议及参数要求\t7\n3.2 关键字说明\t7\n3.3 接口数据规范\t10\n3.4 路由与VPN\t17\n4 网管与安全配置\t23\n4.1 关闭未使用端口与协议\t23\n4.2 SYSLOG配置\t23\n4.3 SNMP配置\t24\n4.4 管理口配置\t25\n4.5 配置本地帐号\t25\n4.6 配置远程访问\t25\n4.7 NTP配置\t26\n\n\n\n'}
Title: 中国移动网络云局数据规范（华为草稿）
Content length: 779
#####################################

In [10]:
import docx
from collections import defaultdict
from docx.document import Document
from docx.table import Table
from docx.oxml.shape import CT_Picture
from docx.oxml.shared import qn
import base64
import io
from langchain.docstore.document import Document
def get_document_title(doc):
    # Try to get the title from document properties
    title = doc.core_properties.title
    
    # If title is not set in properties, use the first paragraph if it's in a title style
    if not title:
        if doc.paragraphs:
            first_paragraph = doc.paragraphs[0]
            if first_paragraph.style.name.lower() in ['title', 'heading 1']:
                title = first_paragraph.text
    
    # If still no title, use a default
    if not title:
        title = "Untitled Document"
    
    return title

def extract_docx_content(file_path):
    doc = docx.Document(file_path)
    content = defaultdict(list)
    heading_stack = ["Root"]  # Start with a default root heading
    current_heading = "Root"
    document_title = get_document_title(doc)
    heading_stack = [document_title]
    current_heading = document_title
    
    for element in doc.element.body:
        if element.tag.endswith('p'):
            paragraph = docx.text.paragraph.Paragraph(element, doc)
            if paragraph.style and paragraph.style.name.startswith('Heading'):
                heading_level = int(paragraph.style.name[-1])
                
                # Adjust the heading stack based on the current heading level
                while len(heading_stack) > heading_level:
                    heading_stack.pop()
                
                if len(heading_stack) < heading_level:
                    # If we jumped to a deeper level, fill in with empty headings
                    heading_stack.extend([""] * (heading_level - len(heading_stack)))
                
                heading_stack[heading_level - 1] = paragraph.text
                current_heading = " > ".join(filter(None, heading_stack))
            else:
                content[current_heading].append({"type": "text", "content": paragraph.text})
        elif element.tag.endswith('tbl'):
            table = Table(element, doc)
            table_data = []
            for row in table.rows:
                table_data.append([cell.text for cell in row.cells])
            content[current_heading].append({"type": "table", "content": table_data})
        elif element.tag.endswith('r'):
            for run in element.findall(".//"+qn('w:drawing')):
                blip = run.find(".//"+qn('a:blip'))
                if blip is not None:
                    image_rid = blip.get(qn('r:embed'))
                    image_part = doc.part.related_parts[image_rid]
                    image_bytes = image_part.blob
                    image_base64 = base64.b64encode(image_bytes).decode()
                    content[current_heading].append({"type": "image", "content": image_base64})
    content[current_heading] = 
    return content

def format_output(content, chunk_size):
    output = []
    for heading, elements in content.items():
        block = {
            "metadata": {"title": heading},
            "content": []
        }
        for element in elements:
            if element['type'] == 'text':
                block['content'].append(f"{element['content']}")
            elif element['type'] == 'table':
                table_str = "\n".join([", ".join(row) for row in element['content']])
                block['content'].append(table_str)
            elif element['type'] == 'image':
                block['content'].append(f"[Base64 encoded image data]")
        block['content'] = "\n".join(block['content'])
        
        if len(block['content']) > chunk_size:
            chunks = [block['content'][i:i+chunk_size] for i in range(0, len(block['content']), chunk_size)]
            for chunk in chunks:
                output.append({"metadata": {"title": heading}, "content": chunk})
        else:    
            output.append(block)
    return output

def main(file_path, chunk_size):
    extracted_content = extract_docx_content(file_path)
    formatted_output = format_output(extracted_content, chunk_size)
    
    for block in formatted_output:
        print(block)
        print(f"Title: {block['metadata']['title']}")
        print(f"Content length: {len(block['content'])}")
        print("#" * 100)
    
    return formatted_output

if __name__ == "__main__":
    file_path = "/root/llm/Langchain-Chatchat-master/notebooks/docx-test/中国移动集中网络云承载网网管&业支CE局数据配置规范初稿（华为）V4.0.docx"
    formatted_output = main(file_path, chunk_size=8094)

{'metadata': {'title': '中国移动网络云局数据规范（华为草稿）'}, 'content': '\n\n\n\n中国移动网络云承载网网管&业支CE局数据配置规范（华为）\n\n\n\n\n\n\n\n\n\n\n版本号：4.0\n\n\n\n\n\n文档说明：本规范为中国移动网络云承载网信令CE局数据配置指导性规范。\n编号说明：\n（本文档编号依照以下原则进行：\nx.y.z    x: 大版本更新，文档结构变化\n                       y: 具体方案的更改，部分内容的修正)\n文档修改历史\n版本, 更新日期, 修改, 核定, 更新说明, 文档状态, 文档状态\n1.0, 2020-03-21, 杨应雄, 赖小勤, 征求意见稿, 初稿, 初稿\n1.1, 2020-07-08, 王永刚, 赖小勤, 删除上行接口carrier up-hold-time , 删除上行接口carrier up-hold-time , \n1.3, 2021-03, 于潇, 王永刚, 3.4.3 OSPF协议增加上行AR对接BGP和上行AR对接OSPF两种配置方式\n3.3.1删除CE-AR互联延时UP 5000, , \n4.0, 2023-04, 王永刚, , 4.4 SNMP配置将认证模式和加密算法修改为安全性更高的sha2-512和aes256, , \n, , , , , , \n\n\n\n目   录\n目   录\t4\n1 移动电信云承载网CE组网概述\t5\n1.1 设计内容\t5\n1.2 目标组网\t5\n2 软硬件版本说明\t6\n3 局数据配置规范\t7\n3.1 协议及参数要求\t7\n3.2 关键字说明\t7\n3.3 接口数据规范\t10\n3.4 路由与VPN\t17\n4 网管与安全配置\t23\n4.1 关闭未使用端口与协议\t23\n4.2 SYSLOG配置\t23\n4.3 SNMP配置\t24\n4.4 管理口配置\t25\n4.5 配置本地帐号\t25\n4.6 配置远程访问\t25\n4.7 NTP配置\t26\n\n\n\n'}
Title: 中国移动网络云局数据规范（华为草稿）
Content length: 779
#####################################

In [18]:
import docx
from collections import defaultdict
from docx.document import Document
from docx.table import Table
from docx.oxml.shape import CT_Picture
from docx.oxml.shared import qn
import base64
import io
from langchain.docstore.document import Document


def extract_docx_content(file_path):
    doc = docx.Document(file_path)
    content = defaultdict(list)
    current_heading = "No Heading"
    
    for element in doc.element.body:
        if element.tag.endswith('p'):
            paragraph = docx.text.paragraph.Paragraph(element, doc)
            if paragraph.style and paragraph.style.name.startswith('Heading'):
                current_heading = paragraph.text
            else:
                content[current_heading].append({"type": "text", "content": paragraph.text})
        elif element.tag.endswith('tbl'):
            table = Table(element, doc)
            table_data = []
            for row in table.rows:
                table_data.append([cell.text for cell in row.cells])
            content[current_heading].append({"type": "table", "content": table_data})
        elif element.tag.endswith('r'):
            for run in element.findall(".//"+qn('w:drawing')):
                blip = run.find(".//"+qn('a:blip'))
                if blip is not None:
                    image_rid = blip.get(qn('r:embed'))
                    image_part = doc.part.related_parts[image_rid]
                    image_bytes = image_part.blob
                    image_base64 = base64.b64encode(image_bytes).decode()
                    content[current_heading].append({"type": "image", "content": image_base64})
    
    return content

def format_output(content,chunk_size):
    output = []
    for heading, elements in content.items():
        block = {
            "metadata": {"title": heading},
            "content": []
        }
        for element in elements:
            if element['type'] == 'text':
                # block['content'].append(f"Text: {element['content']}")
                block['content'].append(f"{element['content']}")

            elif element['type'] == 'table':
                # table_str = "Table:\n" + "\n".join([", ".join(row) for row in element['content']])
                table_str = "\n".join([", ".join(row) for row in element['content']])
                block['content'].append(table_str)
            elif element['type'] == 'image':
                # block['content'].append(f"Image: [Base64 encoded image data]")
                block['content'].append(f"[Base64 encoded image data]")

        block['content'] = "\n".join(block['content'])
        temp_chunk = []
        if len(block['content'])>chunk_size:
            for i in range(len(block)):
                if i+chunk_size < len(block):
                    temp_chunk.append(block['content'][i,i+chunk_size])
                else:
                    temp_chunk.append(block['content'][i:])
                i = i+chunk_size
            for chunk in temp_chunk:
                # print(chunk)
                output.append({"metadata": {"title": heading},"content": chunk})
        else:    
            output.append(block)
    return output

def main(file_path,chunk_size):
    extracted_content = extract_docx_content(file_path)
    formatted_output = format_output(extracted_content,chunk_size)
    
    # Print the output (you can modify this to save to a file if needed)
    for block in formatted_output:
        print(block)
        # print(f"Title: {block['metadata']['title']}")
        # print(len(f"Content:\n{block['content']}\n"))
        # print(f"Content:\n{block['content']}\n")
        print("#"*100)
    return formatted_output
if __name__ == "__main__":
    file_path = "/root/llm/Langchain-Chatchat-master/notebooks/docx-test/中国移动集中网络云承载网网管&业支CE局数据配置规范初稿（华为）V4.0.docx"  # Replace with your actual file path
    formatted_output = main(file_path,chunk_size=8094)

{'metadata': {'title': 'No Heading'}, 'content': '\n\n\n\n中国移动网络云承载网网管&业支CE局数据配置规范（华为）\n\n\n\n\n\n\n\n\n\n\n版本号：4.0\n\n\n\n\n\n文档说明：本规范为中国移动网络云承载网信令CE局数据配置指导性规范。\n编号说明：\n（本文档编号依照以下原则进行：\nx.y.z    x: 大版本更新，文档结构变化\n                       y: 具体方案的更改，部分内容的修正)\n文档修改历史\n版本, 更新日期, 修改, 核定, 更新说明, 文档状态, 文档状态\n1.0, 2020-03-21, 杨应雄, 赖小勤, 征求意见稿, 初稿, 初稿\n1.1, 2020-07-08, 王永刚, 赖小勤, 删除上行接口carrier up-hold-time , 删除上行接口carrier up-hold-time , \n1.3, 2021-03, 于潇, 王永刚, 3.4.3 OSPF协议增加上行AR对接BGP和上行AR对接OSPF两种配置方式\n3.3.1删除CE-AR互联延时UP 5000, , \n4.0, 2023-04, 王永刚, , 4.4 SNMP配置将认证模式和加密算法修改为安全性更高的sha2-512和aes256, , \n, , , , , , \n\n\n\n目   录\n目   录\t4\n1 移动电信云承载网CE组网概述\t5\n1.1 设计内容\t5\n1.2 目标组网\t5\n2 软硬件版本说明\t6\n3 局数据配置规范\t7\n3.1 协议及参数要求\t7\n3.2 关键字说明\t7\n3.3 接口数据规范\t10\n3.4 路由与VPN\t17\n4 网管与安全配置\t23\n4.1 关闭未使用端口与协议\t23\n4.2 SYSLOG配置\t23\n4.3 SNMP配置\t24\n4.4 管理口配置\t25\n4.5 配置本地帐号\t25\n4.6 配置远程访问\t25\n4.7 NTP配置\t26\n\n\n\n'}
###########################################################################################

In [17]:
from langchain.docstore.document import Document

docs = []
for block in formatted_output:
    # print(type(block['metadata']['title']))
    # print(block)
    docs.append(Document(page_content=block['metadata']['title']+"\n"+block['content'], metadata={"head1":block['metadata']['title']}))

In [18]:
docs

[Document(page_content='No Heading\n\n\n\n\n中国移动网络云承载网网管&业支CE局数据配置规范（华为）\n\n\n\n\n\n\n\n\n\n\n版本号：4.0\n\n\n\n\n\n文档说明：本规范为中国移动网络云承载网信令CE局数据配置指导性规范。\n编号说明：\n（本文档编号依照以下原则进行：\nx.y.z    x: 大版本更新，文档结构变化\n                       y: 具体方案的更改，部分内容的修正)\n文档修改历史\n版本, 更新日期, 修改, 核定, 更新说明, 文档状态, 文档状态\n1.0, 2020-03-21, 杨应雄, 赖小勤, 征求意见稿, 初稿, 初稿\n1.1, 2020-07-08, 王永刚, 赖小勤, 删除上行接口carrier up-hold-time , 删除上行接口carrier up-hold-time , \n1.3, 2021-03, 于潇, 王永刚, 3.4.3 OSPF协议增加上行AR对接BGP和上行AR对接OSPF两种配置方式\n3.3.1删除CE-AR互联延时UP 5000, , \n4.0, 2023-04, 王永刚, , 4.4 SNMP配置将认证模式和加密算法修改为安全性更高的sha2-512和aes256, , \n, , , , , , \n\n\n\n目   录\n目   录\t4\n1 移动电信云承载网CE组网概述\t5\n1.1 设计内容\t5\n1.2 目标组网\t5\n2 软硬件版本说明\t6\n3 局数据配置规范\t7\n3.1 协议及参数要求\t7\n3.2 关键字说明\t7\n3.3 接口数据规范\t10\n3.4 路由与VPN\t17\n4 网管与安全配置\t23\n4.1 关闭未使用端口与协议\t23\n4.2 SYSLOG配置\t23\n4.3 SNMP配置\t24\n4.4 管理口配置\t25\n4.5 配置本地帐号\t25\n4.6 配置远程访问\t25\n4.7 NTP配置\t26\n\n\n\n', metadata={'head1': 'No Heading'}),
 Document(page_content='移动网络云承载网CE组网概述\n设计内容\n中国移动网络云承载网CE设备作为资源池出口设备对

In [21]:
import docx
from collections import defaultdict
from docx.document import Document as DocxDocument
from docx.table import Table
from docx.oxml.shared import qn
import base64
from langchain.docstore.document import Document
from typing import List, Optional

class MyDocxLoader:
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self, chunk_size: Optional[int] = None) -> List[Document]:
        extracted_content = self._extract_docx_content()
        formatted_output = self._format_output(extracted_content, chunk_size)
        for block in formatted_output:
            print(block)
            # print(f"Title: {block['metadata']['title']}")
            # print(len(f"Content:\n{block['content']}\n"))
            # print(f"Content:\n{block['content']}\n")
            print("#"*100)
        return self._create_langchain_documents(formatted_output)

    def _extract_docx_content(self):
        doc = docx.Document(self.file_path)
        content = defaultdict(list)
        current_heading = "No Heading"
        
        for element in doc.element.body:
            if element.tag.endswith('p'):
                paragraph = docx.text.paragraph.Paragraph(element, doc)
                if paragraph.style and paragraph.style.name.startswith('Heading'):
                    current_heading = paragraph.text
                else:
                    content[current_heading].append({"type": "text", "content": paragraph.text})
            elif element.tag.endswith('tbl'):
                table = Table(element, doc)
                table_data = []
                for row in table.rows:
                    table_data.append([cell.text for cell in row.cells])
                content[current_heading].append({"type": "table", "content": table_data})
            elif element.tag.endswith('r'):
                for run in element.findall(".//"+qn('w:drawing')):
                    blip = run.find(".//"+qn('a:blip'))
                    if blip is not None:
                        image_rid = blip.get(qn('r:embed'))
                        image_part = doc.part.related_parts[image_rid]
                        image_bytes = image_part.blob
                        image_base64 = base64.b64encode(image_bytes).decode()
                        content[current_heading].append({"type": "image", "content": image_base64})
        
        return content

    def _format_output(self, content, chunk_size):
        output = []
        for heading, elements in content.items():
            block = {
                "metadata": {"title": heading},
                "content": []
            }
            for element in elements:
                if element['type'] == 'text':
                    block['content'].append(f"{element['content']}")
                elif element['type'] == 'table':
                    table_str = "\n".join([", ".join(row) for row in element['content']])
                    block['content'].append(table_str)
                elif element['type'] == 'image':
                    block['content'].append(f"[Base64 encoded image data]")

            block['content'] = "\n".join(block['content'])
            
            if chunk_size and len(block['content']) > chunk_size:
                for i in range(0, len(block['content']), chunk_size):
                    chunk = block['content'][i:i+chunk_size]
                    output.append({"metadata": {"title": heading}, "content": chunk})
            else:
                output.append(block)
        
        return output

    def _create_langchain_documents(self, formatted_output):
        docs = []
        for block in formatted_output:
            docs.append(Document(
                page_content=f"{block['metadata']['title']}\n{block['content']}", 
                metadata={"head1": block['metadata']['title']}
            ))
        return docs

if __name__ == "__main__":
    file_path = "/root/llm/Langchain-Chatchat-master/notebooks/docx-test/中国移动集中网络云承载网网管&业支CE局数据配置规范初稿（华为）V4.0.docx"
    loader = MyDocxLoader(file_path)
    docs = loader.load(chunk_size=8094)
    print(f"Number of documents: {len(docs)}")
    # print(f"First document content: {docs[0].page_content[:100]}...")  # Print first 100 characters of the first document

{'metadata': {'title': 'No Heading'}, 'content': '\n\n\n\n中国移动网络云承载网网管&业支CE局数据配置规范（华为）\n\n\n\n\n\n\n\n\n\n\n版本号：4.0\n\n\n\n\n\n文档说明：本规范为中国移动网络云承载网信令CE局数据配置指导性规范。\n编号说明：\n（本文档编号依照以下原则进行：\nx.y.z    x: 大版本更新，文档结构变化\n                       y: 具体方案的更改，部分内容的修正)\n文档修改历史\n版本, 更新日期, 修改, 核定, 更新说明, 文档状态, 文档状态\n1.0, 2020-03-21, 杨应雄, 赖小勤, 征求意见稿, 初稿, 初稿\n1.1, 2020-07-08, 王永刚, 赖小勤, 删除上行接口carrier up-hold-time , 删除上行接口carrier up-hold-time , \n1.3, 2021-03, 于潇, 王永刚, 3.4.3 OSPF协议增加上行AR对接BGP和上行AR对接OSPF两种配置方式\n3.3.1删除CE-AR互联延时UP 5000, , \n4.0, 2023-04, 王永刚, , 4.4 SNMP配置将认证模式和加密算法修改为安全性更高的sha2-512和aes256, , \n, , , , , , \n\n\n\n目   录\n目   录\t4\n1 移动电信云承载网CE组网概述\t5\n1.1 设计内容\t5\n1.2 目标组网\t5\n2 软硬件版本说明\t6\n3 局数据配置规范\t7\n3.1 协议及参数要求\t7\n3.2 关键字说明\t7\n3.3 接口数据规范\t10\n3.4 路由与VPN\t17\n4 网管与安全配置\t23\n4.1 关闭未使用端口与协议\t23\n4.2 SYSLOG配置\t23\n4.3 SNMP配置\t24\n4.4 管理口配置\t25\n4.5 配置本地帐号\t25\n4.6 配置远程访问\t25\n4.7 NTP配置\t26\n\n\n\n'}
###########################################################################################

In [20]:
docs

[Document(page_content='No Heading\n\n\n\n\n中国移动网络云承载网网管&业支CE局数据配置规范（华为）\n\n\n\n\n\n\n\n\n\n\n版本号：4.0\n\n\n\n\n\n文档说明：本规范为中国移动网络云承载网信令CE局数据配置指导性规范。\n编号说明：\n（本文档编号依照以下原则进行：\nx.y.z    x: 大版本更新，文档结构变化\n                       y: 具体方案的更改，部分内容的修正)\n文档修改历史\n版本, 更新日期, 修改, 核定, 更新说明, 文档状态, 文档状态\n1.0, 2020-03-21, 杨应雄, 赖小勤, 征求意见稿, 初稿, 初稿\n1.1, 2020-07-08, 王永刚, 赖小勤, 删除上行接口carrier up-hold-time , 删除上行接口carrier up-hold-time , \n1.3, 2021-03, 于潇, 王永刚, 3.4.3 OSPF协议增加上行AR对接BGP和上行AR对接OSPF两种配置方式\n3.3.1删除CE-AR互联延时UP 5000, , \n4.0, 2023-04, 王永刚, , 4.4 SNMP配置将认证模式和加密算法修改为安全性更高的sha2-512和aes256, , \n, , , , , , \n\n\n\n目   录\n目   录\t4\n1 移动电信云承载网CE组网概述\t5\n1.1 设计内容\t5\n1.2 目标组网\t5\n2 软硬件版本说明\t6\n3 局数据配置规范\t7\n3.1 协议及参数要求\t7\n3.2 关键字说明\t7\n3.3 接口数据规范\t10\n3.4 路由与VPN\t17\n4 网管与安全配置\t23\n4.1 关闭未使用端口与协议\t23\n4.2 SYSLOG配置\t23\n4.3 SNMP配置\t24\n4.4 管理口配置\t25\n4.5 配置本地帐号\t25\n4.6 配置远程访问\t25\n4.7 NTP配置\t26\n\n\n\n', metadata={'head1': 'No Heading'}),
 Document(page_content='移动网络云承载网CE组网概述\n设计内容\n中国移动网络云承载网CE设备作为资源池出口设备对

# 最新版代码

In [48]:
import docx
from collections import defaultdict
from docx.table import Table
from docx.oxml.shared import qn
import base64
from langchain.docstore.document import Document

class MyDocxLoader:
    def __init__(self, file_path, chunk_size=8094):
        self.file_path = file_path
        self.chunk_size = chunk_size

    def __get_document_title(self, doc):
        title = doc.core_properties.title
        if not title:
            if doc.paragraphs:
                first_paragraph = doc.paragraphs[0]
                if first_paragraph.style.name.lower() in ['title', 'heading 1']:
                    title = first_paragraph.text
        if not title:
            title = ""
        return title

    def __extract_docx_content(self):
        doc = docx.Document(self.file_path)
        content = defaultdict(list)
        document_title = self.__get_document_title(doc)
        heading_stack = [document_title]
        current_heading = document_title

        for element in doc.element.body:
            if element.tag.endswith('p'):
                paragraph = docx.text.paragraph.Paragraph(element, doc)
                if paragraph.style and paragraph.style.name.startswith('Heading'):
                    heading_level = int(paragraph.style.name[-1])

                    while len(heading_stack) > heading_level:
                        heading_stack.pop()

                    if len(heading_stack) < heading_level:
                        heading_stack.extend([""] * (heading_level - len(heading_stack)))

                    heading_stack[heading_level - 1] = paragraph.text
                    current_heading = document_title + " > "+" > ".join(filter(None, heading_stack))
                else:
                    content[current_heading].append({"type": "text", "content": paragraph.text})
            elif element.tag.endswith('tbl'):
                table = Table(element, doc)
                table_data = []
                for row in table.rows:
                    table_data.append([cell.text for cell in row.cells])
                content[current_heading].append({"type": "table", "content": table_data})
            elif element.tag.endswith('r'):
                for run in element.findall(".//"+qn('w:drawing')):
                    blip = run.find(".//"+qn('a:blip'))
                    if blip is not None:
                        image_rid = blip.get(qn('r:embed'))
                        image_part = doc.part.related_parts[image_rid]
                        image_bytes = image_part.blob
                        image_base64 = base64.b64encode(image_bytes).decode()
                        content[current_heading].append({"type": "image", "content": image_base64})
        return content

    def __format_output(self, content):
        output = []
        for heading, elements in content.items():
            block = {
                "metadata": {"title": heading},
                "content": []
            }
            for element in elements:
                if element['type'] == 'text':
                    block['content'].append(f"{element['content']}")
                elif element['type'] == 'table':
                    table_str = "\n".join([", ".join(row) for row in element['content']])
                    block['content'].append(table_str)
                elif element['type'] == 'image':
                    block['content'].append(f"[Base64 encoded image data]")
            block['content'] = "\n".join(block['content'])

            if len(block['content']) > self.chunk_size:
                chunks = [block['content'][i:i+self.chunk_size] for i in range(0, len(block['content']), self.chunk_size)]
                for chunk in chunks:
                    output.append({"metadata": {"title": heading}, "content": chunk})
            else:
                output.append(block)
        return output

    def load(self):
        extracted_content = self.__extract_docx_content()
        formatted_output = self.__format_output(extracted_content)
        docs = [Document(page_content=block['content'], metadata=block['metadata']) for block in formatted_output]
        return docs

# Example usage
if __name__ == "__main__":
    loader = MyDocxLoader("./docx-test/DPtech ADX3000系列应用交付平台典型配置案例(1).docx")
    documents = loader.load()
    for doc in documents:
        print(doc)
        # print(f"Title: {doc.metadata['title']}")
        # print(f"Content: {doc.page_content}")
        print("#" * 100)


page_content='\n\n\n‘\n\n\n\n\n\n\n\n\n\n\n\n' metadata={'title': ''}
####################################################################################################
page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n手册版本：5.2\n软件版本：ADX3000-S211C012D004P03PATCH07.bin\n发布日期：2020-05-07\n\n\nDPtech ADX3000 系列应用交付平台典型配置案例.docx\n\n\n' metadata={'title': ' > DPtech ADX3000 系列应用交付平台典型配置案例'}
####################################################################################################
page_content='\nCopyright ©2008-2020 杭州迪普科技股份有限公司版权所有，保留一切权利。\n非经本公司书面许可，任何单位和个人不得擅自摘抄、复制本书内容的部分或全部，并不得 以任何形式传播。\n\n\n为杭州迪普科技股份有限公司的商标。\n对于本手册中出现的其他所有商标或注册商标，由各自的所有人拥有。\n\n\n\n由于产品版本升级或其他原因，本手册内容有可能变更。杭州迪普科技股份有限公司保留在 没有任何通知或者提示的情况下对本手册的内容进行修改的权利。本手册仅作为使用指导， 杭州迪普科技股份有限公司尽全力在本手册中提供准确的信息，但是杭州迪普科技股份有限 公司并不确保手册内容完全没有错误，本手册中的所有陈述、信息和建议也不构成任何明示 或暗示的担保。\n\n\n\n\n\n\n\n杭州迪普科技股份有限公司\n地址：杭州市滨江区通和路 68 号中财大厦 6 层\n邮编：310051\n网址：www.dptech.com\n技术论坛：http://forum.dptech.com\n7x24 小时技术服务热线：400-6100-598\n\n\nDPtech ADX3

In [18]:
'''
Author       : your name
Date         : 2024-07-30 10:37:21
LastEditors  : your name
LastEditTime : 2024-07-30 11:42:31
FilePath     : /notebooks/test_doc_split.py
Description  : 
Copyright 2024 OBKoro1, All Rights Reserved. 
2024-07-30 10:37:21
'''
import os
from docx import Document
import docx
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import traceback
import re
def split_text_by_sentense(text):
    sentences = re.split(r"([！？。；])", text)
    output = []
    for sent in sentences:
        if sent in "！？；。" and output:
            output[-1] += sent
        else:
            output.append(sent)
    return output
    
def clean_spaces(t: str):
    try:
        t = re.sub(r"\t+|+", " ", t)
        return re.sub(r"\s+", " ", t.strip())
    except Exception:
        print("Error in re.sub")
        print(t)

def docx_general_preprocess(
    path,
    pics_folder: str,
    **kwargs,
):
    """
    做如下预处理：
    保存图片 在原文中定位
    转化表格，保存为excel并转文字 在原文中定位？ # 这个先不做
    输出：list[list]: docx每一段[doc每一句+图片 or docx表格]
    """
    # print("process docx", get_tables)
    get_tables = kwargs.get("get_tables", False)
    # get_image_ocr = kwargs.get("get_image_ocr", False)
    if not path.endswith(".docx"):
        raise ValueError("文件类型必须是docx")
        # return False
    filename = os.path.split(path)[-1]  # 有后缀文件名

    total_docx_content = []
    total_image_para = {}  # 图片名称+段落号

    # if not pics_folder:
    #     pics_folder = f"./pics_{os.path.splitext(filename)[0]}"
    # if os.path.exists(pics_folder):
    #     shutil.rmtree(pics_folder)
    #     os.mkdir(pics_folder)
    # else:
    #     os.mkdir(pics_folder)
    # print(f"pictures will be saved to: {pics_folder}")

    # 打开docx文件
    doc = docx.Document(path)
    blocks = iter_block_items(doc)

    pic_count = 0
    try:
        for i, block in enumerate(blocks):
            paragraph_content = []
            para_imgs = []
            p = block

            if isinstance(block, Paragraph):
                # print(p.text)1
                # p_text = p.text.replace("\n", "  ")
                p_text = p.text
                print(p_text)
                print("#"*10)
                # if p.text:
                texts = split_text_by_sentense(p_text)
                for t in texts:
                    if t.strip():
                        t = clean_spaces(t)
                        paragraph_content.append(t)
                # 获取本段图片
                # images = get_picture_list(doc, [p])
                # if images:
                #     for image in images:
                #         image_saved_name = get_docx_imgs_totext(
                #             image, pic_count, pics_folder, **kwargs
                #         )
                #         # print(image_saved_name)
                #         if image_saved_name:
                #             # new_doc.paragraphs[-1].add_run(image_saved_label)
                #             # para_imgs.append(IMAGE_LABEL_TEMP.format(image_saved_name))
                #             paragraph_content.append(image_saved_name)
                #             pic_count += 1




                        # print(para_imgs)
                # if not p.text.strip() and not para_imgs:
                #     continue
                # insert_text_by_toc_with_update(p.text)


            # elif isinstance(block, Table):
            #     if get_tables:
            #         # print(1)
            #         # 提取表格文字和图片，转换成text
            #         tbl_text, tbl_pic_count = tbl2_text_withpics(
            #             doc, block, pics_folder=pics_folder
            #         )
            #         pic_count += tbl_pic_count
            #         # print(tbl_text)
            #         # new_doc.paragraphs[-1].add_run("; ".join(tbl_text))
            #         paragraph_content = tbl_text
            # # print(para_imgs)
            if paragraph_content:  # Ensure non-empty content is appended
                total_docx_content.append(paragraph_content)
            # total_docx_content.append(paragraph_content)
            # if para_imgs:
            #     total_image_para[i] = para_imgs
                
        # print(total_image_para)
        return total_docx_content, total_image_para
    except Exception as e:
        traceback.print_exc()
        err_msg = str(e)
        print(f"文档预处理失败！Error message: {err_msg}")
        return [], {}



def iter_block_items(parent):
    """
    Yield each paragraph and table child within *parent*, in document order.
    Each returned value is an instance of either Table or Paragraph. *parent*
    would most commonly be a reference to a main Document object, but
    also works for a _Cell object, which itself can contain paragraphs and tables.
    """
    if isinstance(parent, docx.document.Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")
    blocks = []
    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            # yield Paragraph(child, parent)
            blocks.append(Paragraph(child, parent))
        elif isinstance(child, CT_Tbl):
            # yield Table(child, parent)
            blocks.append(Table(child, parent))
        # elif isinstance(child, CT_SectPr):
        # #     blocks.append(CT_SectPr)
        #     blocks.append(child)
        # adding picture
        # elif isinstance(child, CT_Picture):
        #     blocks.append(Image(child, parent))

    return blocks


blocks = docx_general_preprocess("/root/llm/Langchain-Chatchat-master/notebooks/docx-test/docx_test.docx","./pic")


##########

##########

##########

##########

##########

##########

##########

##########

##########

##########

##########

##########

##########

##########

##########
目  录
##########
1 工具概述	1
##########
1.1 简介	1
##########
1.2 操作流程	1
##########
2 检查前准备	3
##########
3 安装工具	5
##########
3.1 场景说明	5
##########
3.1.1 安装场景说明	5
##########
3.1.2 巡检场景说明	5
##########
3.2 安装MrsCare	6
##########
3.3 对接环境	7
##########
3.3.1 添加对接环境	7
##########
3.3.1.1 手动添加环境信息	8
##########
3.3.1.2 通过配置文件导入环境信息	10
##########
3.3.2 批量添加客户端路径	11
##########
4 检查过程	13
##########
4.1 产品支持的任务类型、版本和用户	13
##########
4.2 执行日常巡检检查	14
##########
4.3 执行升级前检查	15
##########
4.4 执行补丁前检查	16
##########
4.5 执行自动化验收	17
##########
4.6 执行质量验收巡检	18
##########
4.7 执行深度巡检	19
##########
4.8 检查结果	19
##########
4.8.1 查看检查结果	19
##########
4.8.2 检查项修复	20
##########
4.8.3 重复巡检	24
##########
4.9 多环境场景下的巡检工具部署	25
##########
4.9.1 部署方案	25
##########
4.9.2 部署指导	25
##########
4.10 邮箱配置	27
##########
4.11 巡检帮助链接	28
##########
5 卸载工具	30
##

In [15]:
blocks

([['目 录'],
  ['1 工具概述 1'],
  ['1.1 简介 1'],
  ['1.2 操作流程 1'],
  ['2 检查前准备 3'],
  ['3 安装工具 5'],
  ['3.1 场景说明 5'],
  ['3.1.1 安装场景说明 5'],
  ['3.1.2 巡检场景说明 5'],
  ['3.2 安装MrsCare 6'],
  ['3.3 对接环境 7'],
  ['3.3.1 添加对接环境 7'],
  ['3.3.1.1 手动添加环境信息 8'],
  ['3.3.1.2 通过配置文件导入环境信息 10'],
  ['3.3.2 批量添加客户端路径 11'],
  ['4 检查过程 13'],
  ['4.1 产品支持的任务类型、版本和用户 13'],
  ['4.2 执行日常巡检检查 14'],
  ['4.3 执行升级前检查 15'],
  ['4.4 执行补丁前检查 16'],
  ['4.5 执行自动化验收 17'],
  ['4.6 执行质量验收巡检 18'],
  ['4.7 执行深度巡检 19'],
  ['4.8 检查结果 19'],
  ['4.8.1 查看检查结果 19'],
  ['4.8.2 检查项修复 20'],
  ['4.8.3 重复巡检 24'],
  ['4.9 多环境场景下的巡检工具部署 25'],
  ['4.9.1 部署方案 25'],
  ['4.9.2 部署指导 25'],
  ['4.10 邮箱配置 27'],
  ['4.11 巡检帮助链接 28'],
  ['5 卸载工具 30'],
  ['5.1 卸载SysChecker 30'],
  ['5.2 卸载MrsCare 30'],
  ['6 附录 32'],
  ['6.1 如何修改巡检任务的超时时间 32'],
  ['6.2 检查项修复指导 32'],
  ['6.2.1 CDL 32'],
  ['6.2.1.1 检查CDL服务和实例状态 32'],
  ['6.2.1.2 检查CDL告警信息 33'],
  ['6.2.1.3 检查CDL服务状态 33'],
  ['6.2.2 ClickHouse 33'],
  ['6.2.2.1 检查ClickHouse告警信息 33'],
  ['6.2.2.2 检查Click